<div dir="rtl">
<font size=5 color="crimson">
نکته: خواهشمند است این نوتبوک در کولب اجرا شود نه در ژوپیتر محلی، چون آدرس درایور در سلول ۴ بر اساس تنظیمات کولب است و وقتی در رایانه محلی اجرا شود ارور می‌دهد.
</font>
</div>

In [ ]:
# Cell 1

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.I7oKXoYYvZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.DdMBwIeOVo/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.DkSZphTth5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Ign:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://deb.debian.o

In [ ]:
# Cell 2

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service

import pandas as pd
import numpy as np

In [ ]:
# Cell 3

def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [ ]:
# Cell 4

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

driver.get("http://anthony.sogang.ac.kr/ThreeKingdoms/")
links = [link.get_attribute("href") for link in driver.find_elements(By.XPATH, "//a[@href]") if link.text.startswith("Chapter")]
by_chapter = {}
for index in range(len(links)):
    link = links[index]
    driver.get(link)
    chapter_text = driver.find_element(By.XPATH, "/html/body/div[1]").text.strip()
    chapter_text = chapter_text[chapter_text.find("Chapter " + str(index + 1)): ]
    title = chapter_text[find_nth(chapter_text, "\n", 1) + 1 : find_nth(chapter_text, "\n", 3)]
    text = chapter_text[find_nth(chapter_text, "\n", 3) + 1 : ]
    by_chapter["Chapter " + str(index + 1)] = {"title": title, "text": text}
    print(title)
    print("".join(["-"] * 100))
    driver.back()
# look at how chapter 46 is stored as an example
print(by_chapter["Chapter 46"])

Three Heroes Swear Brotherhood In The Peach Garden;
One Victory Shatters The Rebels In Battlegrounds.
----------------------------------------------------------------------------------------------------
Zhang Fei Whips The Government Officer;
He Jin Plots To Kill The Eunuchs.
----------------------------------------------------------------------------------------------------
In Wenming Garden, Dong Zhuo Denounces Ding Yuan;
With Red Hare, Li Su Bribes Lu Bu.
----------------------------------------------------------------------------------------------------
The Deposition Of The Emperor: Prince Of Chenliu Becomes Emperor;
Schemes Against Dong Zhuo: Cao Cao Presents A Sword.
----------------------------------------------------------------------------------------------------
Cao Cao Appeals To The Powerful Lords;
The Three Brothers Fight Against Lu Bu.
----------------------------------------------------------------------------------------------------
Burning The Capital, Dong Zhuo Commi

In [ ]:
# Cell 5
import json

with open("Sanguo Yanyi.json","w") as f:
    json.dump(by_chapter, f)


In [ ]:
# Cell 6
driver.get("https://en.wikipedia.org/wiki/Lists_of_people_of_the_Three_Kingdoms")
links = [link.get_attribute("href") for link in driver.find_elements(By.XPATH, "//a[@href]") if link.text.startswith("List of people of the Three Kingdoms")
and link.text.find("(I)") == -1
and link.text.find("(U)") == -1
and link.text.find("(V)") == -1]
# print("\n".join(links))
story_characters = pd.DataFrame(columns=['Name', 'Courtesy name', 'Birth year', 'Death year', 'Ancestral home (present-day location)', 'Role', 'Allegiance', 'Previous allegiance(s)', 'Notes'])
for index in range(len(links)):
    link = links[index]
    driver.get(link)
    character_table = driver.find_element(By.CSS_SELECTOR, "#mw-content-text > div.mw-parser-output > table")
    character_df = pd.DataFrame(pd.read_html(character_table.get_attribute('outerHTML'))[0])
    print(character_df.shape)
    print(character_df.columns.tolist())
    print(character_df.head())
    print("".join(["-"] * 100))
    story_characters = pd.concat([story_characters, character_df],axis=0)
    driver.back()

story_characters.to_excel("SanguoYanyiChars.xlsx", encoding='utf-8')


(4, 9)
['Name', 'Courtesy name', 'Birth year', 'Death year', 'Ancestral home (present-day location)', 'Role', 'Allegiance', 'Previous allegiance(s)', 'Notes']
          Name  Courtesy name  Birth year  Death year  \
0      Agui 阿貴            NaN         NaN       212.0   
1  Aluoduo 阿羅多            NaN         NaN         NaN   
2  Aluopan 阿羅槃            NaN         NaN         NaN   
3       Awu 阿騖            NaN         NaN         NaN   

  Ancestral home (present-day location)                 Role       Allegiance  \
0        Xingguo (Qin'an County, Gansu)        Tribal leader               Di   
1                                   NaN        Tribal leader          Xianbei   
2          Youbeiping (Tangshan, Hebei)                Noble  Wuhuan, Cao Wei   
3                                   NaN  Xun You's concubine              NaN   

   Previous allegiance(s)                                    Notes  
0                     NaN  Weilue annotation in Sanguozhi vol. 30.  
1          

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
